In [1]:
from main import *
import main as cdt
from qiskit.utils import QuantumInstance
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
%precision 2

'%.2f'

In [5]:
IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend  = provider.get_backend('ibmq_quito')

ibmqfactory.load_account:WARNING:2022-03-09 18:31:47,426: Credentials are already in use. The existing account in the session will be replaced.


In [6]:
import qiskit.tools.jupyter
backend

<IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>

In [7]:
shots = 2**10
quantum_instance = QuantumInstance( Aer.get_backend('aer_simulator'), shots=shots)
resampling = 0

## Gate set tomography

In [8]:
# gate set times 
# 1q 11.2s
# 2q 22.1s
# 3q 33.1s

In [9]:
gst = tomographic_gate_set_tomography(1)
circuits_gst = gst.circuits()
results  = quantum_instance.execute( circuits_gst )                                            

In [12]:
operators_gst = gst.fit( results,resampling=resampling )

## QND Measurement tomography single qubit

In [ ]:
# times
# 1q 0.7s
# 2q 1.4s
# 3q 2.2s

In [17]:
mpt = measurement_process_tomography(1, 3)
circuits_mpt = mpt.circuits()
results  = quantum_instance.execute( circuits_mpt )

In [18]:
operators_mpt = mpt.fit( results, resampling=resampling )    

## QND measurement tomography two qubits

In [ ]:
# times
# 1p 1m 6.8s
# 2p 2m 13.2s
# 3p 3M 20.9s

In [23]:
mpt = measurement_process_tomography(2, 1)
circuits_mpt = mpt.circuits()
results  = quantum_instance.execute( circuits_mpt )

In [24]:
operators_mpt = mpt.fit(results, resampling=resampling, out=1 )

## QND measurement tomography two qubits + gate set tomography

In [ ]:
# time
# 1p 1m 6.7s

In [18]:
pairs = 1
num_qubits = 2*pairs
mpt_2 = measurement_process_tomography(2, pairs)
gst_2 = tomographic_gate_set_tomography(num_qubits)
circuits_mpt_2 = mpt_2.circuits() + gst_2.circuits()
results  = quantum_instance.execute( circuits_mpt_2 )

In [20]:
gate_set = gst_2.fit(results)

In [21]:
states, measurements = gst_2.gateset2spam( gate_set )

In [22]:
operators_mpt_2 = mpt_2.fit(results, resampling=resampling, out=1, gate_set=[states, measurements] )

## Device QND measurement tomography

In [23]:
# time
# 5-qubits 3m 31.8s
# 
# currently 5*12s + 5*1s + 2*(1m+7s) = 199s = 3.31m = 3m 19s
# optimal   12s + 1s + 2*(1m+7s) = 147s = 2.24m = 2m 15s


In [36]:
# extrapolation 65, in a 8-cores devices

# 65q --> 9 loops

# 3 groups of  26, 21, 26 = 73 ---> 10 loops
#                              ---> 8*3=24 loops

# currently  9*12s + 9*1s + 10*(1m+7s)   = 787s  = 13.12m = 13m 7s
# optimal    65*12s + 65*1s + 24*(1m+7s) = 2453s = 40.88m = 40m 53s


In [9]:
dpmt = device_process_measurement_tomography( backend )
circuits_dpmt = dpmt.circuits()
results  = quantum_instance.execute( circuits_dpmt )

In [10]:
operators_dpmt = dpmt.fit( results, resampling=resampling, paralell=True )

In [12]:
parall_qubits = dpmt._parall_qubits

In [13]:
len(parall_qubits)

3

In [16]:
parall_qubits[2]

[(1, 2)]